# Creating a synth drumkit using ipytone and jupyter widgets

## UNDER CONSTRUCTION

In [ ]:
import ipytone
import ipywidgets as widgets

import matplotlib.pyplot as plt
# %matplotlib

In [3]:
channel1 = ipytone.Channel(pan=-0.5).to_destination()
filtr = ipytone.Filter(frequency=100)
synth = ipytone.MonoSynth(volume=-7).chain(filtr, channel1)

channel2 = ipytone.Channel(pan=0.2, channel_count=2).to_destination()
delay = ipytone.PingPongDelay(delay_time="16n", feedback=0.2)
perc_synth = ipytone.MembraneSynth(volume=-10).chain(delay, channel2)

In [4]:
lfo = ipytone.LFO(frequency="4m", min=100, max=10_000)
lfo.connect(filtr.frequency).start()

lfo2 = ipytone.LFO(frequency="8n", min=-200, max=200, type="triangle")
lfo2.connect(perc_synth.detune).start()

perc_synth.pitch_decay = 0.02
delay.wet.value = 0.1
lfo2.amplitude.value = 0.6

In [5]:
def clb(time, note):
    synth.trigger_attack_release(note, 0.2, time=time)
    perc_synth.trigger_attack_release(note, 0.05, time=time)

sequence = ipytone.Sequence(
    callback=clb,
    events=["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"],
    subdivision="16n",
)

In [6]:
ipytone.get_transport().start()
sequence.start()

Sequence(loop=True, loop_start=0.0, loop_end=0.0)

In [7]:
ipytone.get_transport().pause()

Transport()

In [8]:
import asyncio

async def melody():
    """8x2 measures melody"""
    m2 = ipytone.get_transport().bpm.value / 60 * 2

    # 1 ---
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "A4"]
    await asyncio.sleep(m2)
    # 2 ---
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "B4"]
    await asyncio.sleep(m2)
    # 3 ---
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "C4"]
    await asyncio.sleep(m2)
    # 4 ---
    sequence.events = ["A1", "A2", None, "A0", "E5", None, "E4", "A4"]
    await asyncio.sleep(m2)
    # 5 ---
    sequence.events = ["A1", None, None, "A0", "E5", None, None, None]
    await asyncio.sleep(m2)
    # 6 ---
    synth.portamento = 0.08
    filtr.q.ramp_to(6, "4m")
    lfo2.amplitude.ramp_to(1, "4m")
    await asyncio.sleep(m2)
    # 7 ---
    sequence.events = ["A1", "A2", None, "A0", "E5", None, "G5", "A4"]
    await asyncio.sleep(m2)
    # 8 ---
    delay.wet.ramp_to(0.8, "2m")
    synth.portamento = 0.1
    await asyncio.sleep(m2)
    # 1 ---
    synth.portamento = 0
    filtr.q.ramp_to(1, "4n")
    delay.wet.ramp_to(0.1, "4n")
    lfo.amplitude.ramp_to(0.1, "4n")
    sequence.events = ["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"]


In [9]:
loop = asyncio.get_event_loop()
loop.create_task(melody());

In [10]:
def create_channel(node):
    # panner
    pan = widgets.FloatSlider(
        value=node.pan.value, min=-1, max=1,
        layout=widgets.Layout(width="200px")
    )
    widgets.jslink((pan, "value"), (node.pan, "value"))
    
    # solo / mute buttons
    solo = widgets.ToggleButton(value=False, description="Solo")
    mute = widgets.ToggleButton(value=False, description="Mute")

    def node_solo(change):
        node.solo = change['new']
        
    def node_mute(change):
        node.mute = change["new"]
        
    solo.observe(node_solo, names='value')
    mute.observe(node_mute, names='value')
    
    # fader
    fader = widgets.FloatSlider(
        value=0, min=-30, max=4, orientation="vertical"
    )
    widgets.jslink((fader, "value"), (node.volume, "value"))

    # L/R VU meters
    vu_left = widgets.FloatProgress(
        min=0, max=0.4, orientation="vertical"
    )
    vu_right = widgets.FloatProgress(
        min=0, max=0.4, orientation="vertical"
    )
    split = ipytone.Split()
    node.connect(split)
    meter_left = ipytone.Meter(normal_range=True)
    split.connect(meter_left, 0, 0)
    meter_left.schedule_jsdlink((vu_left, "value"), transport=True)
    meter_right = ipytone.Meter(normal_range=True)
    split.connect(meter_right, 1, 0)
    meter_right.schedule_jsdlink((vu_right, "value"), transport=True)
    
    # layout
    fader_vus = widgets.HBox([fader, vu_left, vu_right])
    return widgets.VBox([pan, solo, mute, fader_vus])


widgets.HBox([create_channel(channel1), create_channel(channel2)])

In [11]:
start_stop_button = widgets.ToggleButton(description="Start/Stop")

def start_stop_seq(change):
    if change['new']:
        ipytone.get_transport().start()
        print("bange")
    else:
        ipytone.get_transport().stop()
        
    
start_stop_button.observe(start_stop_seq, names="value")

start_stop_button

ToggleButton(value=False, description='Start/Stop')

---

In [12]:
osc = ipytone.Oscillator(volume=-5)

osc.output

Volume(volume=Param(value=-5.0, units='decibels'), mute=False)

In [13]:
osc.connect(ipytone.destination)


Oscillator()

In [14]:
# Disconnect to add the filter
osc.disconnect(ipytone.destination)

#create the filter
filtr = ipytone.Filter(type="highpass", frequency=1000)

# connects in chain oscillator -> filter -> destination
osc.chain(filtr, ipytone.destination)


Oscillator()

In [15]:
import ipywidgets

freq_slider = ipywidgets.FloatSlider(
    value=440,
    min=100,
    max=1000,
    step=1,
)

type_dropdown = ipywidgets.Dropdown(
    options=['sine', 'square', 'sawtooth', 'triangle'],
    value='sine',
)

toggle_play_button = ipywidgets.ToggleButton(
    value=False,
    description="Start/Stop"
)

ipywidgets.jslink((freq_slider, 'value'), (osc.frequency, 'value'))
ipywidgets.link((type_dropdown, 'value'), (osc, 'type'))

def start_stop_osc(change):
    if change['new']:
        osc.start()
    else:
        osc.stop()

toggle_play_button.observe(start_stop_osc, names='value')

ipywidgets.VBox([freq_slider, type_dropdown, toggle_play_button])

In [16]:
channel_one = ipytone.Channel(channel_count=1).to_destination()

delay = ipytone.PingPongDelay(delay_time="16n", feedback=0.2)

perc_synth = ipytone.MembraneSynth(volume=-10).chain(delay, channel_one)

lfo = ipytone.LFO(frequency="8n", min=-200, max=200, type="triangle")
lfo.connect(perc_synth.detune).start()


perc_synth.pitch_decay = 0.02
delay.wet.value = 0.1
lfo.amplitude.value = 0.6

In [17]:
perc_synth.trigger_attack_release(440.0, 0.5)

MembraneSynth()

In [18]:
def clb(time, note):
    #synth.trigger_attack_release(note, 0.2, time=time)
    perc_synth.trigger_attack_release(note, 0.05, time=time)

sequence = ipytone.Sequence(
    callback=clb,
    events=["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"],
    subdivision="16n",
)

In [19]:
ipytone.get_transport().start()
sequence.start()

Sequence(loop=True, loop_start=0.0, loop_end=0.0)

In [20]:
ipytone.get_transport().stop()

Transport()

# oberserving state from python
- https://ipytone.readthedocs.io/en/latest/observe_link.html#observing-state-from-python

In [21]:
synth = ipytone.Synth().to_destination()


output = ipywidgets.Output()

def print_new_value(change):
    widget = change["owner"].observed_widget

    with output:
        print(widget, " : ", change["new"])

output

Output()

In [22]:
synth.oscillator.frequency.schedule_observe(
    print_new_value,
    update_interval=0.5,
    name="value",
    transport=False,
)

In [23]:
synth.trigger_attack_release("C3", 1)
synth.trigger_attack_release("A3", 1, time="+1")

Synth()

In [24]:
# set a longer envelope attack
synth.envelope.attack = 1

In [25]:
synth.envelope.schedule_observe(print_new_value, update_interval=0.1)


In [26]:
synth.trigger_attack_release("C3", 2)


Synth()

In [27]:
synth.oscillator.frequency.schedule_unobserve(print_new_value)
synth.envelope.schedule_unobserve(print_new_value)

In [28]:
synth.oscillator.frequency.schedule_observe(
    print_new_value,
    update_interval=3,
    observe_time=True
)

In [29]:
output.clear_output()
output

Output(outputs=({'name': 'stdout', 'text': "Signal(value=440.0, units='frequency')  :  130.8127826502993\nAmpl…

In [30]:
synth.oscillator.frequency.schedule_unobserve(print_new_value)


In [31]:
progress = ipywidgets.FloatProgress(value=0, min=0, max=1)

link = synth.envelope.schedule_jsdlink((progress, "value"))

progress

FloatProgress(value=0.0, max=1.0)

In [32]:
synth.trigger_attack_release("C3", 2)

Synth()

In [ ]:
link.unlink()

In [48]:
# Create the output for an analyzer
# print(plt.style.available)
# Makes the plot a little less epilepsy inducing on a dark background
plt.style.use('dark_background')

plot_output = ipywidgets.Output(layout=ipywidgets.Layout(height="500px"))

def plot_change(change):
    plot_output.clear_output()
    with plot_output:
        plt.plot(change["new"])
        plt.show()

plot_output

Output(layout=Layout(height='500px'))

In [41]:
analyser = ipytone.Analyser(type="waveform")
synth.connect(analyser)

Synth()

In [42]:
analyser.schedule_observe(plot_change, update_interval=0.5)


In [49]:
synth.trigger_attack_release("C4", 2)


Synth()

In [44]:
analyser.schedule_unobserve(plot_change)
analyser.dispose()

Analyser(disposed=True)

In [108]:
synth.dispose()

Synth(disposed=True)

---